In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim 
from torchvision import datasets, transforms 

In [0]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3,padding = 1) 
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding =1 )
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout2d(0.1)
        self.conv3 = nn.Conv2d(32, 16, 1)
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout2d(0.1)
        self.conv5 = nn.Conv2d(32, 32, 3,padding = 1)
        self.bn5 = nn.BatchNorm2d(32)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.conv7 = nn.Conv2d(32, 10, 1) 
      
     
    def forward(self, x): 
        x = self.drop1(self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x))))))))
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.conv7(x))
        x = self.avgpool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x,dim=1)

In [3]:
!pip install torchsummary 
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 32, 28, 28]           4,640
       BatchNorm2d-4           [-1, 32, 28, 28]              64
         MaxPool2d-5           [-1, 32, 14, 14]               0
         Dropout2d-6           [-1, 32, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]             528
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 32, 14, 14]           4,640
      BatchNorm2d-10           [-1, 32, 14, 14]              64
        MaxPool2d-11             [-1, 32, 7, 7]               0
           Conv2d-12             [-1, 32, 7, 7]           9,248
      BatchNorm2d-13             [-1, 32, 7, 7]              64
           Conv2d-14             [-1, 1

In [4]:
torch.manual_seed(1) 
batch_size = 8
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader( 
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) 
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/7500 [00:00<?, ?it/s]


Test set: Average loss: 0.0487, Accuracy: 9844/10000 (98.440%)



  0%|          | 0/7500 [00:00<?, ?it/s]


Test set: Average loss: 0.0367, Accuracy: 9878/10000 (98.780%)



  0%|          | 0/7500 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0190, Accuracy: 9939/10000 (99.390%)



  0%|          | 0/7500 [00:00<?, ?it/s]


Test set: Average loss: 0.0161, Accuracy: 9942/10000 (99.420%)



loss=0.0012736320495605469 batch_id=7499: 100%|██████████| 7500/7500 [01:01<00:00, 121.17it/s]



Test set: Average loss: 0.0186, Accuracy: 9931/10000 (99.310%)

